<a href="https://colab.research.google.com/github/chaechaeguri/10th_Seminar/blob/main/%5B%EC%BF%A0%EA%B8%80%5D_kaggle_%EB%8C%80%ED%9A%8C_%EC%9B%90%ED%95%AB_%EC%A0%95%EA%B7%9C%ED%99%94x_cat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

수정을 해봤습니다.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **데이터 로드 및 분리**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 훈련데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백팩/train.csv')
train.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723,112.15875
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537,68.88056
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760,39.17320
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220,80.60793
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338,86.02312


In [ ]:

# 테스트데이터 불러오기
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백팩/test.csv')
test.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
1,300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
2,300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
3,300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
4,300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [ ]:
# 타겟 데이터 분리
train_target=train['Price']
train_target.head()
train_input=train.drop('Price',axis=1)
train_input.head()

,id,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,0,Jansport,Leather,Medium,7.0,Yes,No,Tote,Black,11.611723
1,1,Jansport,Canvas,Small,10.0,Yes,Yes,Messenger,Green,27.078537
2,2,Under Armour,Leather,Small,2.0,Yes,No,Messenger,Red,16.643760
3,3,Nike,Nylon,Small,8.0,Yes,No,Messenger,Green,12.937220
4,4,Adidas,Canvas,Medium,1.0,Yes,Yes,Messenger,Green,17.749338


In [ ]:
train_input=train_input.drop('id',axis=1)
test=test.drop('id',axis=1)

# **데이터 전처리**

In [ ]:
train_input['Waterproof'] = train_input['Waterproof'].map({'Yes': 1, 'No': 0})
test['Waterproof'] = test['Waterproof'].map({'Yes': 1, 'No': 0})

train_input['Laptop Compartment'] = train_input['Laptop Compartment'].map({'Yes': 1, 'No': 0})
test['Laptop Compartment'] = test['Laptop Compartment'].map({'Yes': 1, 'No': 0})

size_mapping = {'Small': 1, 'Medium': 2, 'Large': 3}
train_input['Size'] = train['Size'].map(size_mapping)
test['Size'] = test['Size'].map(size_mapping)

In [ ]:
train_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 290295 non-null  object 
 1   Material              291653 non-null  object 
 2   Size                  293405 non-null  float64
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    292556 non-null  float64
 5   Waterproof            292950 non-null  float64
 6   Style                 292030 non-null  object 
 7   Color                 290050 non-null  object 
 8   Weight Capacity (kg)  299862 non-null  float64
dtypes: float64(5), object(4)
memory usage: 20.6+ MB


In [ ]:
# 확인용
train_input.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
0,Jansport,Leather,2.0,7.0,1.0,0.0,Tote,Black,11.611723
1,Jansport,Canvas,1.0,10.0,1.0,1.0,Messenger,Green,27.078537
2,Under Armour,Leather,1.0,2.0,1.0,0.0,Messenger,Red,16.643760
3,Nike,Nylon,1.0,8.0,1.0,0.0,Messenger,Green,12.937220
4,Adidas,Canvas,2.0,1.0,1.0,1.0,Messenger,Green,17.749338


# **결측값 처리**

In [ ]:
train_input.isnull().sum()

,0
Brand,9705
Material,8347
Size,6595
Compartments,0
Laptop Compartment,7444
Waterproof,7050
Style,7970
Color,9950
Weight Capacity (kg),138


**수치형 변수 결측값 처리**

In [ ]:
#Size
train_input['Size'].mode()

,Size
0,2.0


In [ ]:
train_input['Size'].value_counts(normalize=True) * 100

,proportion
Size,
2.0,34.732196
3.0,33.620081
1.0,31.647722


In [ ]:
train_input['Size']=train_input['Size'].fillna(2)
test['Size']=test['Size'].fillna(2)

In [ ]:
#laptop compartment
train_input['Laptop Compartment'].value_counts(normalize=True) * 100

,proportion
Laptop Compartment,
1.0,50.705506
0.0,49.294494


In [ ]:
train_input['Laptop Compartment']=train_input['Laptop Compartment'].fillna(0)
test['Laptop Compartment']=test['Laptop Compartment'].fillna(0)

In [ ]:
#waterproof
train_input['Waterproof'].value_counts(normalize=True) * 100

,proportion
Waterproof,
1.0,50.546851
0.0,49.453149


In [ ]:
train_input['Waterproof']=train_input['Waterproof'].fillna(0)
test['Waterproof']=test['Waterproof'].fillna(0)

In [ ]:
#weight capacity
train_input['Weight Capacity (kg)'].mode()

,Weight Capacity (kg)
0,5.0


In [ ]:
train_input['Weight Capacity (kg)'].describe()

,Weight Capacity (kg)
count,299862.000000
mean,18.029994
std,6.966914
min,5.000000
25%,12.097867
50%,18.068614
75%,24.002375
max,30.000000


In [ ]:
# 중앙값으로 대체
train_input['Weight Capacity (kg)']=train_input['Weight Capacity (kg)'].fillna(18.07)
test['Weight Capacity (kg)']=test['Weight Capacity (kg)'].fillna(18.07)

**범주형 변수 결측값 처리**

In [ ]:
#범주형 변수 결측값 처리
categorical=['Brand', 'Style', 'Material', 'Color']
for cat in categorical:
  train_input[cat]=train_input[cat].fillna(train_input[cat].mode()[0])
  test[cat]=test[cat].fillna(test[cat].mode()[0])

In [ ]:
#결측치 처리 확인
train_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 300000 non-null  object 
 1   Material              300000 non-null  object 
 2   Size                  300000 non-null  float64
 3   Compartments          300000 non-null  float64
 4   Laptop Compartment    300000 non-null  float64
 5   Waterproof            300000 non-null  float64
 6   Style                 300000 non-null  object 
 7   Color                 300000 non-null  object 
 8   Weight Capacity (kg)  300000 non-null  float64
dtypes: float64(5), object(4)
memory usage: 20.6+ MB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200000 entries, 0 to 199999
Data columns (total 9 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Brand                 200000 non-null  object 
 1   Material              200000 non-null  object 
 2   Size                  200000 non-null  float64
 3   Compartments          200000 non-null  float64
 4   Laptop Compartment    200000 non-null  float64
 5   Waterproof            200000 non-null  float64
 6   Style                 200000 non-null  object 
 7   Color                 200000 non-null  object 
 8   Weight Capacity (kg)  200000 non-null  float64
dtypes: float64(5), object(4)
memory usage: 13.7+ MB


**범주형 변수 변환**

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#레이블인코딩
cols = [ "Brand", "Style", "Material", "Color"]
for col in cols:
  le = LabelEncoder()
  train_input[col] = le.fit_transform(train_input[col])
  test[col]=le.transform(test[col])

In [ ]:
#원핫인코딩
cols = [ "Brand", "Style", "Material", "Color"]
train_input = pd.get_dummies(train_input, columns=cols).astype(int)
test = pd.get_dummies(test, columns=cols).astype(int)

# **정규화 / 표준화**

In [ ]:
train_input.describe()

,Size,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,...,Material_Canvas,Material_Leather,Material_Nylon,Material_Polyester,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
count,300000.000000,300000.000000,300000.000000,300000.00000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,...,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,2.019290,5.443590,0.494473,0.49359,17.464243,0.232607,0.186920,0.191120,0.189380,0.199973,...,0.226680,0.244720,0.235343,0.293257,0.154593,0.160323,0.167600,0.154620,0.205467,0.157397
std,0.798724,2.890766,0.499970,0.49996,6.946155,0.422494,0.389848,0.393184,0.391811,0.399981,...,0.418685,0.429922,0.424214,0.455256,0.361517,0.366906,0.373511,0.361543,0.404043,0.364175
min,1.000000,1.000000,0.000000,0.00000,5.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,3.000000,0.000000,0.00000,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,5.000000,0.000000,0.00000,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,8.000000,1.000000,1.00000,24.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,10.000000,1.000000,1.00000,30.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train_input[['Weight Capacity (kg)']])
train_input[['Weight Capacity (kg)']] = scaler.transform(train_input[['Weight Capacity (kg)']])
test[['Weight Capacity (kg)']] = scaler.transform(test[['Weight Capacity (kg)']])

scaler = MinMaxScaler()
scaler.fit(train_input[['Compartments']])
train_input[['Compartments']] = scaler.transform(train_input[['Compartments']])
test[['Compartments']] = scaler.transform(test[['Compartments']])

In [ ]:
train_input.describe()

,Size,Compartments,Laptop Compartment,Waterproof,Weight Capacity (kg),Brand_Adidas,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,...,Material_Canvas,Material_Leather,Material_Nylon,Material_Polyester,Color_Black,Color_Blue,Color_Gray,Color_Green,Color_Pink,Color_Red
count,300000.000000,300000.000000,300000.000000,300000.00000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,...,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,2.019290,0.493732,0.494473,0.49359,0.498570,0.232607,0.186920,0.191120,0.189380,0.199973,...,0.226680,0.244720,0.235343,0.293257,0.154593,0.160323,0.167600,0.154620,0.205467,0.157397
std,0.798724,0.321196,0.499970,0.49996,0.277846,0.422494,0.389848,0.393184,0.391811,0.399981,...,0.418685,0.429922,0.424214,0.455256,0.361517,0.366906,0.373511,0.361543,0.404043,0.364175
min,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.222222,0.000000,0.00000,0.280000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,0.444444,0.000000,0.00000,0.520000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,0.777778,1.000000,1.00000,0.760000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# **catboost 모델 적용**

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.3 MB/s eta 0:00:00


# **랜덤서치 원핫 정규화x**

In [ ]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV

# CatBoost 모델 정의
model = CatBoostRegressor(verbose=1000)  # 출력 줄이기

# 탐색할 파라미터 범위 설정
param = {
    'depth': [4, 5, 6, 7, 8],
    'iterations': [100, 300, 500, 700],
    'learning_rate': np.linspace(0.01, 0.1, 10),
    'loss_function': ['RMSE']
    }

# RandomizedSearchCV 실행
random_search = RandomizedSearchCV(model, param_distributions=param, cv=5, scoring="neg_root_mean_squared_error",pre_dispatch="2*n_jobs", n_iter=15, n_jobs=4, random_state=42)
random_search.fit(train_input, train_target)

0:	learn: 39.0389747	total: 90.2ms	remaining: 1m 3s
699:	learn: 38.9992224	total: 25.1s	remaining: 0us


RandomizedSearchCV(cv=5,
                   estimator=<catboost.core.CatBoostRegressor object at 0x7b285e0497d0>,
                   n_iter=15, n_jobs=4,
                   param_distributions={'depth': [4, 5, 6, 7, 8],
                                        'iterations': [100, 300, 500, 700],
                                        'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ]),
                                        'loss_function': ['RMSE']},
                   random_state=42, scoring='neg_root_mean_squared_error')

In [ ]:

print("최적의 파라미터:", random_search.best_params_)
print("최적의 RMSE:", -random_search.best_score_)  # RMSE는 음수로 나오므로 -를 붙여서 출

최적의 파라미터: {'loss_function': 'RMSE', 'learning_rate': 0.01, 'iterations': 700, 'depth': 4}
최적의 RMSE: 39.0130217577362


In [ ]:
results = random_search.cv_results_

# RMSE(음수로 저장됨) 기준으로 정렬
sorted_indices = np.argsort(results["mean_test_score"])  # 오름차순 정렬

# 최악의 조합 (가장 낮은 점수)
worst_idx = sorted_indices[0]  # 첫 번째 인덱스 (가장 낮은 점수)
worst_params = results["params"][worst_idx]
worst_score = results["mean_test_score"][worst_idx]
print(f"🔥 최악의 조합: {worst_params}, Score: {worst_score:.4f}")

🔥 최악의 조합: {'loss_function': 'RMSE', 'learning_rate': 0.09000000000000001, 'iterations': 700, 'depth': 7}, Score: -39.1485


In [ ]:
#모델 설정
model = CatBoostRegressor(
    iterations=700,        # 트리 개수
    learning_rate=0.01,     # 학습률
    depth=4,               # 트리 깊이
    loss_function='RMSE',   # 평균 절대 오차(MAE) 최소화
    verbose=200,          # 학습 과정 출력 (100번마다)
    random_seed=42,
    eval_metric='RMSE'
)

In [ ]:
#교차검증
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

train_input, val_input, train_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

cv_score=cross_val_score(model, train_input, train_target, scoring='neg_root_mean_squared_error', cv=5)

0:	learn: 39.0675860	total: 22.3ms	remaining: 15.6s
200:	learn: 39.0416438	total: 3.85s	remaining: 9.56s
400:	learn: 39.0322925	total: 9.77s	remaining: 7.29s
600:	learn: 39.0251361	total: 13.6s	remaining: 2.24s
699:	learn: 39.0218275	total: 15.5s	remaining: 0us
0:	learn: 39.0653046	total: 21.6ms	remaining: 15.1s
200:	learn: 39.0389108	total: 4.42s	remaining: 11s
400:	learn: 39.0290908	total: 9.88s	remaining: 7.37s
600:	learn: 39.0216474	total: 13.7s	remaining: 2.25s
699:	learn: 39.0180827	total: 15.6s	remaining: 0us
0:	learn: 39.0459074	total: 19.5ms	remaining: 13.6s
200:	learn: 39.0205315	total: 6.03s	remaining: 15s
400:	learn: 39.0108016	total: 9.84s	remaining: 7.34s
600:	learn: 39.0033265	total: 13.7s	remaining: 2.25s
699:	learn: 39.0000978	total: 15.8s	remaining: 0us
0:	learn: 39.0654197	total: 48.4ms	remaining: 33.9s
200:	learn: 39.0392703	total: 5.6s	remaining: 13.9s
400:	learn: 39.0298757	total: 9.37s	remaining: 6.99s
600:	learn: 39.0226248	total: 13.2s	remaining: 2.17s
699:	lea

In [ ]:
print(f"Cross-validation scores: {cv_score}")
print(f"score: {cv_score.mean():.4f}")

Cross-validation scores: [-39.01641682 -39.03213024 -39.10503708 -39.03161382 -39.0114448 ]
score: -39.0393


In [ ]:
#모델 학습
model.fit(train_input,train_target)

0:	learn: 39.0627824	total: 70.6ms	remaining: 49.4s
200:	learn: 39.0376283	total: 4.84s	remaining: 12s
400:	learn: 39.0292263	total: 9.59s	remaining: 7.15s
600:	learn: 39.0229275	total: 16.5s	remaining: 2.72s
699:	learn: 39.0201415	total: 18.9s	remaining: 0us


In [ ]:
# Train RMSE 계산

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

train_pred = model.predict(train_input)
train_rmse = np.sqrt(mean_squared_error(train_target, train_pred))

print(f"Train RMSE: {train_rmse:.4f}")

Train RMSE: 39.0201


In [ ]:
model.score(train_input,train_target)

0.00219464476493636

In [ ]:
#제출

# 예측 수행
predictions = model.predict(test)

# sample_submission 형식에 맞게 DataFrame 생성
submission = pd.DataFrame({
    "ID": test.index,  # ID가 따로 있는 경우 test["ID"] 사용
    "Price": predictions
})

# CSV 저장 (Kaggle 제출용)
submission.to_csv("submission.csv", index=False)

# sample_submission 파일 불러오기
sample_submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백팩/sample_submission.csv')

# 예측 (예시)
predictions = model.predict(test)

# sample_submission의 형식을 유지하면서 예측값 반영
submission = sample_submission.copy()
submission.iloc[:, 1] = predictions  # 두 번째 컬럼(예측값) 변경

# CSV 저장
submission.to_csv("submission_onehot_정규화0(1).csv", index=False)

# 파일 다운로드
from google.colab import files
files.download("submission_onehot_정규화0(1).csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>